In [321]:
%matplotlib inline

#必要なライブラリを import 
from sklearn import datasets
import numpy as np
from sklearn.svm import SVC
import pandas as pd
import glob as glob
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import StandardScaler 
from sklearn.multiclass import OneVsRestClassifier

files =  glob.glob('AdvancedTouchSensing/Processing_collect_data/data/nectorwater/*.csv')
voltage = pd.DataFrame()

for f in files:
    col_names = [ 'c{0:02d}'.format(i) for i in range(161) ]
    df = pd.read_csv(f,names = col_names)
    voltage = pd.concat([voltage, df],join="outer")
    
voltage = voltage.reset_index()
if type(voltage["c00"][0]) is np.float64 :
    voltage = voltage.drop(axis=1,labels="c160")
for index,row in voltage.iterrows():
    voltage.loc[index] = voltage.loc[index].fillna(method="ffill")

X = voltage.drop(labels = "index",axis=1)
y = voltage["index"]

if type(voltage["c00"][0]) is not np.float64 :
    X = voltage.drop(labels = "c00",axis=1).drop(labels = "index",axis = 1)
    y = voltage["c00"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None )
model = SVC(kernel="linear", random_state=None)
classifier = OneVsRestClassifier(model)
model.fit(X_train,y_train)
classifier.fit(X_train,y_train)

from sklearn.metrics import accuracy_score

pred_train = model.predict(X_train)
accuracy_train = accuracy_score(y_train, pred_train)
print('OvOトレーニングデータに対する正解率： %.2f' % accuracy_train)

pred_test = model.predict(X_test)
accuracy_test = accuracy_score(y_test, pred_test)
print('OvOテストデータに対する正解率： %.2f' % accuracy_test)

pred_train = classifier.predict(X_train)
accuracy_train = accuracy_score(y_train, pred_train)
print('OvRトレーニングデータに対する正解率： %.2f' % accuracy_train)

pred_test = classifier.predict(X_test)
accuracy_test = accuracy_score(y_test, pred_test)
print('OvRテストデータに対する正解率： %.2f' % accuracy_test)



OvOトレーニングデータに対する正解率： 1.00
OvOテストデータに対する正解率： 0.93
OvRトレーニングデータに対する正解率： 1.00
OvRテストデータに対する正解率： 0.85


In [323]:
# python側を先に起動
# Processingのserver.ip()を確認．

import socket
import pandas as pd
import numpy as np

host = "192.168.10.4" #Processingで立ち上げたサーバのIPアドレス
port = 10001       #Processingで設定したポート番号

HOST = ''                 # Symbolic name meaning all available interfaces
PORT = 10002              # Arbitrary non-privileged port

col_names = [ 'c{0:02d}'.format(i) for i in range(160) ]
voltageArray = pd.DataFrame(columns=col_names)
voltageArray.loc[0] = np.nan
readCounter = 0
readStrList = []

with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.bind((HOST, PORT))
    s.listen(1)
    conn, addr = s.accept()
    with conn:
        print('Connected by', addr)
        while True:
            data = conn.recv(1024)
            if not data: break
            readStr = data.decode()
#                 Voltageデータを形成する
#                 print(readStr)#string
            readStrList.extend(readStr.split("/"))
            lastStr = readStrList[-1]
            if lastStr == "finished" :
#                 print(readStrList)
                readStrList = [i for i in readStrList if len(i) != 0]
#                 print(readStrList)
                for i in range(len(readStrList) - 1):
                    voltageArray['c{0:02d}'.format(i)] = float(readStrList[i])
#                     print(voltageArray)
                voltageArray.loc[0] = voltageArray.loc[0].fillna(method="ffill")
#                     print(voltageArray)
                state = model.predict(voltageArray)[0]
                print(state)
                socket_client = socket.socket(socket.AF_INET, socket.SOCK_STREAM) #オブジェクトの作成
                socket_client.connect((host, port))                               #サーバに接続
                socket_client.send(state.encode('utf-8')) #データを送信 Python3
#                     初期化
                readStrList = []
                voltageArray = pd.DataFrame(columns=col_names)
                voltageArray.loc[0] = np.nan
                readStrList = []
                    

Connected by ('127.0.0.1', 59815)
Nothing
Nothing
Nothing
Nothing
Nothing
Nothing
Nothing
Nothing
Nothing
Nothing
Nothing
Nothing
Nothing
Nothing
Nothing
Nothing
Nothing
Nothing
Nothing
Nothing
Nothing
Nothing
Nothing
Nothing
Nothing
Nothing
Nothing
Nothing
Nothing
Nothing
Nothing
Nothing
Nothing
Nothing
Nothing
Nothing
Nothing
Near
Near
Near
Near
Near
Near
Near
Near
Near
Nothing
Nothing
Nothing
Nothing
Nothing
Nothing
Nothing
Nothing
Nothing
Nothing
Nothing
Near
Near
Near
Near
Near
Near
Near
Near
Nothing
Nothing
Nothing
Nothing
Near
Near
Near
Near
Near
Near
Near
Near
Near
Near
Near
Near
Near
Near
Near
Near
Near
Near
Near
Near
Near
Near
Near
Near
Near
Near
Near
One-finger-Touch
One-finger-Touch
One-finger-Touch
One-finger-Touch
One-finger-Touch
One-finger-Touch
One-finger-Touch
One-finger-Touch
One-finger-Touch
One-finger-Touch
One-finger-Touch
One-finger-Touch
One-finger-Touch
One-finger-Touch
One-finger-Touch
One-finger-Touch
One-finger-Touch
One-finger-Touch
Two-finger-Touch
Two-fin

Two-finger-Touch
Two-finger-Touch
Two-finger-Touch
One-finger-Touch
Near
Near
Nothing
Nothing
Nothing
Nothing
Nothing
Nothing
Nothing
Nothing
Nothing
Nothing
Nothing
Nothing
Nothing
Nothing
Nothing
Nothing
Nothing
Nothing
Nothing
Nothing
Nothing
Nothing
Nothing
Nothing
Nothing
Nothing
Nothing
Nothing
Nothing
Nothing
Nothing
Nothing
Nothing
Nothing
Nothing
Nothing
Nothing
Nothing
Nothing
Near
Near
Near
Near
Nothing
Nothing
Nothing
Nothing
Near
Near
Near
Nothing
Nothing
Nothing
Nothing
Nothing
Nothing
Nothing
Nothing
Nothing
Nothing
Nothing
Nothing
Nothing
Nothing
Near
Near
Near
Near
Near
Near
Near
Nothing
Nothing
Nothing
Nothing
Nothing
Nothing
Nothing
Nothing
Nothing
Nothing
Nothing
Nothing
Nothing
Nothing
Nothing
Nothing
One-finger-Touch
One-finger-Touch
One-finger-Touch
One-finger-Touch
One-finger-Touch
One-finger-Touch
One-finger-Touch
One-finger-Touch
One-finger-Touch
One-finger-Touch
One-finger-Touch
One-finger-Touch
One-finger-Touch
One-finger-Touch
One-finger-Touch
One-finger-To

ConnectionRefusedError: [Errno 61] Connection refused

In [310]:
%matplotlib inline

#必要なライブラリを import 
from sklearn import datasets
import numpy as np
from sklearn.svm import SVC
import pandas as pd
import glob as glob
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import StandardScaler 
from sklearn.multiclass import OneVsRestClassifier

files =  glob.glob('AdvancedTouchSensing/Processing_collect_data/data/nectorwater/*.csv')
voltage = pd.DataFrame()

for f in files:
    col_names = [ 'c{0:02d}'.format(i) for i in range(161) ]
    df = pd.read_csv(f,names = col_names)
    voltage = pd.concat([voltage, df],join="outer")
    
voltage = voltage.reset_index()
if type(voltage["c00"][0]) is np.float64:
    voltage = voltage.drop(axis=1,labels="c160")
for index,row in voltage.iterrows():
    voltage.loc[index] = voltage.loc[index].fillna(method="ffill")

In [315]:
X

,c01,c02,c03,c04,c05,c06,c07,c08,c09,c10,...,c151,c152,c153,c154,c155,c156,c157,c158,c159,c160
0,232.0,198.0,138.0,154.0,135.0,147.0,136.0,144.0,136.0,142.0,...,222.0,223.0,218.0,222.0,222.0,224.0,225.0,228.0,227.0,227.0
1,230.0,197.0,138.0,153.0,134.0,147.0,136.0,145.0,136.0,144.0,...,222.0,222.0,219.0,222.0,223.0,225.0,225.0,225.0,228.0,228.0
2,240.0,203.0,134.0,152.0,136.0,147.0,136.0,147.0,136.0,143.0,...,222.0,223.0,219.0,221.0,222.0,224.0,224.0,227.0,228.0,228.0
3,239.0,199.0,136.0,155.0,134.0,146.0,136.0,145.0,136.0,145.0,...,220.0,221.0,223.0,226.0,224.0,227.0,229.0,229.0,229.0,229.0
4,239.0,198.0,139.0,154.0,134.0,146.0,136.0,144.0,137.0,143.0,...,221.0,219.0,222.0,222.0,225.0,223.0,226.0,228.0,228.0,228.0
5,231.0,197.0,136.0,152.0,133.0,146.0,137.0,145.0,137.0,143.0,...,222.0,221.0,222.0,223.0,225.0,223.0,225.0,228.0,228.0,228.0
6,232.0,197.0,137.0,154.0,134.0,146.0,136.0,145.0,136.0,143.0,...,227.0,221.0,222.0,219.0,222.0,222.0,224.0,224.0,225.0,229.0
7,242.0,197.0,140.0,152.0,134.0,146.0,137.0,145.0,137.0,143.0,...,222.0,221.0,220.0,221.0,223.0,225.0,223.0,225.0,228.0,228.0
8,237.0,199.0,136.0,154.0,134.0,148.0,136.0,144.0,137.0,143.0,...,222.0,222.0,218.0,223.0,223.0,225.0,223.0,226.0,228.0,228.0
9,232.0,200.0,135.0,153.0,135.0,146.0,135.0,145.0,138.0,143.0,...,222.0,223.0,219.0,222.0,223.0,225.0,224.0,225.0,230.0,230.0


False

In [298]:
type(voltage["c00"][0]) is not np.float64

True

In [139]:
len(model.coef_[0])

160

In [136]:
model.intercept_

array([  0.63235525,  -4.70395986,  -6.20303939,  -4.64755242,
        -1.59184058,  -0.67900474,  -1.01444858,   8.59267543,
        25.63861543, -19.50599777])

str